In [364]:
user_query = "how good is food at Five Guys?"

In [365]:
from typing import Dict, List
from autogen import ConversableAgent
import sys
import math
from dotenv import load_dotenv
import os
from typing import Dict, List

In [366]:
load_dotenv()

api_key = os.environ.get("OPENAI_API_KEY")

In [367]:
def fetch_restaurant_data(restaurant_name: str) -> Dict[str, List[str]]:
    """
    Fetches the reviews for a specific restaurant from the restaurant-reviews.txt file.

    Args:
        restaurant_name: The name of the restaurant.

    Returns:
        A dictionary with the restaurant name as the key and a list of reviews as the value.
    """
    reviews = {}
    try:
        with open("restaurant-data.txt", "r") as file:
            for line in file:
                parts = line.strip().split(". ", 1)
                if len(parts) == 2:
                    name, review = parts
                    if restaurant_name.lower() in name.lower():
                        if name not in reviews:
                            reviews[name] = []
                        reviews[name].append(review)
    except FileNotFoundError:
        print("Error: restaurant-reviews.txt not found.")
        return {}

    return reviews

In [368]:
def calculate_overall_score(
    restaurant_name: str, food_scores: List[int], customer_service_scores: List[int]
) -> Dict[str, float]:
    """
    Calculates the overall score for a restaurant based on food and customer service scores.

    Args:
        restaurant_name: The name of the restaurant.
        food_scores: A list of food scores (1-5).
        customer_service_scores: A list of customer service scores (1-5).

    Returns:
        A dictionary with the restaurant name as the key and the overall score as the value.
    """
    if not food_scores or not customer_service_scores:
        return {restaurant_name: 0.0}

    if len(food_scores) != len(customer_service_scores):
        print(
            "Error: food_scores and customer_service_scores must have the same length."
        )
        return {restaurant_name: 0.0}

    n = len(food_scores)
    sum_of_scores = 0
    for i in range(n):
        sum_of_scores += math.sqrt(food_scores[i] ** 2 * customer_service_scores[i])

    overall_score = (sum_of_scores / (n * math.sqrt(125))) * 10
    return {restaurant_name: round(overall_score, 3)}

In [369]:
def get_data_fetch_agent_prompt(restaurant_query: str) -> str:
    return f"""
    Your task is to fetch restaurant data based on the user's query. 
    The user is asking about: "{restaurant_query}".

    1. Determine the restaurant name the user is asking about. Be robust to variations in phrasing or spelling.
    2. Try to use internet conventional name of the restaurant as user can enter restaurant name in any format. i.e. "applebees", "applebee's", "Applebee's".
    2. Suggest a function call to `fetch_restaurant_data` with the extracted restaurant name as the argument.
    3. Only suggest a function call if you can confidently identify the restaurant name from the user's query.
    4. Return value should be empty if found nothing.
    
    Return a dictionary with the restaurant name as the key and a list of ALL the reviews as the value. (Do not limit to 5 reviews)
    """

In [370]:
def get_review_analysis_agent_prompt() -> str:
    return """
    Your task is to analyze restaurant reviews and extract food and customer service scores.

    You will be given a list of reviews for a specific restaurant. Each review contains adjectives that describe the food and customer service.

    Here are the keywords and their corresponding scores (1-5):

    Food/Service Score | Keywords
    -------------------|-------------------------------------
    1                  | awful, horrible, disgusting
    2                  | bad, unpleasant, offensive
    3                  | average, uninspiring, forgettable
    4                  | good, enjoyable, satisfying
    5                  | awesome, incredible, amazing

    For each review:

    1. Identify the keyword describing the food quality.
    2. Assign a food_score (1-5) based on the keyword.
    3. Identify the keyword describing the customer service quality.
    4. Assign a customer_service_score (1-5) based on the keyword.
    5. Create a dictionary for the review containing:
       - "food_score": the extracted food score
       - "customer_service_score": the extracted customer service score

    Return a list of dictionaries, where each dictionary represents the scores for a single review. (Do not limit to 5 reviews)
    NOTE: output should only contain the list of dictionaries and Restaurant name.
    """

In [371]:
def get_scoring_agent_prompt() -> str:
    return """
    You are the Scoring Agent. Your task is to calculate the overall score for a restaurant based on the provided food and customer service scores.

    You will receive a summary containing the following information:

    - restaurant_name: The name of the restaurant.
    - food_scores: A list of food scores (integers from 1 to 5).
    - customer_service_scores: A list of customer service scores (integers from 1 to 5).

    Your task is to suggest a function call to `calculate_overall_score` with the provided information as arguments. Make sure to pass all necessary arguments correctly.

    The `calculate_overall_score` function will return a dictionary with the restaurant name and the overall score.
    Return me something like this: ## Average food quality at [restaurant_name] is [overall_score]. [WRITE A REASONING with 3-4 bullet points in new line and also write witty comments] ##
    """

In [372]:
entrypoint_agent_system_message = """
    You are the Entry Point Agent, the supervisor responsible for orchestrating the entire process of answering user queries about restaurants.
    You will delegate tasks to specialized agents and ensure that the user's query is answered accurately and efficiently.
    You will initiate conversations with other agents and pass relevant information between them using summaries generated at the end of each conversation.
    You will use function calls to execute actions that other agents suggest.
    Always terminate the conversation when the user's query has been fully answered and the overall score has been calculated.
    """

In [373]:
def _is_termination_message(message) -> bool:
    """Check if the message is a termination message."""
    content = message.get("content", "")
    if content is None:
        return False
    return "TERMINATE" in content.upper()

In [374]:
data_fetch_agent = ConversableAgent(
    "data_fetch_agent",
    system_message=get_data_fetch_agent_prompt(user_query),
    llm_config=llm_config,
    is_termination_msg=_is_termination_message,
)

data_fetch_agent.register_for_llm(
    name="fetch_restaurant_data",
    description="Fetches the reviews for a specific restaurant.",
)(fetch_restaurant_data)

<function __main__.fetch_restaurant_data(restaurant_name: str) -> Dict[str, List[str]]>

In [375]:
review_analysis_agent = ConversableAgent(
    "review_analysis_agent",
    system_message=get_review_analysis_agent_prompt(),
    llm_config=llm_config,
    is_termination_msg=_is_termination_message,
)

In [376]:
scoring_agent = ConversableAgent(
    "scoring_agent",
    system_message=get_scoring_agent_prompt(),
    llm_config=llm_config,
    is_termination_msg=_is_termination_message,
    function_map={"calculate_overall_score": calculate_overall_score},
    
)
scoring_agent.register_for_llm(
    name="calculate_overall_score",
    description="Calculates the overall score for a restaurant based on food and customer service scores.",
)(calculate_overall_score)

<function __main__.calculate_overall_score(restaurant_name: str, food_scores: List[int], customer_service_scores: List[int]) -> Dict[str, float]>

In [377]:
entrypoint_agent = ConversableAgent(
    "entrypoint_agent",
    system_message=entrypoint_agent_system_message,
    llm_config=llm_config,
    is_termination_msg=lambda msg: msg.get("content") is not None
    and "TERMINATE" in msg["content"],
)
entrypoint_agent.register_for_execution(name="fetch_restaurant_data")(
    fetch_restaurant_data
)

entrypoint_agent.register_for_execution(name="calculate_overall_score")(
    calculate_overall_score
)

<function __main__.calculate_overall_score(restaurant_name: str, food_scores: List[int], customer_service_scores: List[int]) -> Dict[str, float]>

In [378]:
result = entrypoint_agent.initiate_chats(
    [
        {
            "recipient": data_fetch_agent,
            "message": f"Find reviews for the restaurant mentioned in this query: {user_query}",
            "max_turns": 2,
            "summary_method": "last_msg",
        },
        {
            "recipient": review_analysis_agent,
            "message": "These are my reviews",
            "max_turns": 1,
            "summary_method": "last_msg",
        },
        {
            "recipient": scoring_agent,
            "message": "You are given array of dictionaries with food_score and customer_service_score",
            "max_turns": 2,
            "summary_method": "reflection_with_llm",
        },
    ]
)


********************************************************************************
Starting a new chat....

********************************************************************************
entrypoint_agent (to data_fetch_agent):

Find reviews for the restaurant mentioned in this query: how good is food at Five Guys?

--------------------------------------------------------------------------------

>>>>>>>> USING AUTO REPLY...
data_fetch_agent (to entrypoint_agent):

***** Suggested tool call (call_RdvAt3jE5neYhltKJEN1Dcbf): fetch_restaurant_data *****
Arguments: 
{"restaurant_name":"Five Guys"}
**************************************************************************************

--------------------------------------------------------------------------------

>>>>>>>> USING AUTO REPLY...

>>>>>>>> EXECUTING FUNCTION fetch_restaurant_data...
entrypoint_agent (to data_fetch_agent):

***** Response from calling tool (call_RdvAt3jE5neYhltKJEN1Dcbf) *****
{"Five Guys": ["Five Guys offers s

In [379]:
from IPython.display import display, Markdown

display(Markdown(result[2].chat_history[-1]["content"]))

## Average food quality at Five Guys is 7.155. 

- The burgers shine with a solid food score of 4, showcasing great taste and customization options.
- The customer service matches the food, also scoring a consistent 4, contributing to a well-rounded dining experience.
- Cleanliness and attentiveness from the staff enhance the satisfaction level.
- Overall, it's a place where the food and service dance in impeccable harmony, ensuring a happy visit.

Why did the burger go to school? Because it wanted to get a little "meat"-er! 🌟🍔